In [1]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *

In [2]:
home_data = spark.read.csv('FileStore/tables/home_data.csv',header = True,inferSchema = True)

In [3]:
display(home_data)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7129300520,20141013T000000,221900,3,1.0,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.721,-122.319,1690,7639
5631500400,20150225T000000,180000,2,1.0,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000,4,3.0,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000,3,2.0,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
7237550310,20140512T000000,1225000,4,4.5,5420,101930,1.0,0,0,3,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
1321400060,20140627T000000,257500,3,2.25,1715,6819,2.0,0,0,3,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
2008000270,20150115T000000,291850,3,1.5,1060,9711,1.0,0,0,3,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
2414600126,20150415T000000,229500,3,1.0,1780,7470,1.0,0,0,3,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
3793500160,20150312T000000,323000,3,2.5,1890,6560,2.0,0,0,3,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [4]:
sqft_living_assembler = VectorAssembler(inputCols=["sqft_living"],outputCol="features")
sqft_living_df= sqft_living_assembler.transform(home_data)
(trainingData, testData) = sqft_living_df.randomSplit([0.8, 0.2],seed = 11L)
sqft_living_lr = LinearRegression(labelCol="price", featuresCol="features")
sqft_living_lr_model = sqft_living_lr.fit(trainingData)
sqft_living_predictions = sqft_living_lr_model.transform(testData)

In [5]:
testData.select(mean('price').alias('price_value')).collect()
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
sqft_rmse = evaluator.evaluate(sqft_living_predictions) 
print("Root Mean Squared Error (RMSE) on test data = %g" % sqft_rmse)

Root Mean Squared Error (RMSE) on test data = 281234

In [6]:
my_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'zipcode']
sales.select(my_features).show()
my_features_assembler = VectorAssembler(inputCols=my_features,outputCol="features")
my_features_df= my_features_assembler.transform(home_data)
(my_features_trainingData, my_features_testData) = my_features_df.randomSplit([0.8, 0.2],seed = 11L)

+--------+---------+-----------+--------+------+-------+
bedrooms|bathrooms|sqft_living|sqft_lot|floors|zipcode|
+--------+---------+-----------+--------+------+-------+
 3| 1.0| 1180| 5650| 1.0| 98178|
 3| 2.25| 2570| 7242| 2.0| 98125|
 2| 1.0| 770| 10000| 1.0| 98028|
 4| 3.0| 1960| 5000| 1.0| 98136|
 3| 2.0| 1680| 8080| 1.0| 98074|
 4| 4.5| 5420| 101930| 1.0| 98053|
 3| 2.25| 1715| 6819| 2.0| 98003|
 3| 1.5| 1060| 9711| 1.0| 98198|
 3| 1.0| 1780| 7470| 1.0| 98146|
 3| 2.5| 1890| 6560| 2.0| 98038|
 3| 2.5| 3560| 9796| 1.0| 98007|
 2| 1.0| 1160| 6000| 1.0| 98115|
 3| 1.0| 1430| 19901| 1.5| 98028|
 3| 1.75| 1370| 9680| 1.0| 98074|
 5| 2.0| 1810| 4850| 1.5| 98107|
 4| 3.0| 2950| 5000| 2.0| 98126|
 3| 2.0| 1890| 14040| 2.0| 98019|
 4| 1.0| 1600| 4300| 1.5| 98103|
 2| 1.0| 1200| 9850| 1.0| 98002|
 3| 1.0| 1250| 9774| 1.0| 98003|
+--------+---------+-----------+--------+------+-------+
only showing top 20 rows

In [7]:
my_features_lr = LinearRegression(labelCol="price", featuresCol="features")
my_features_model = my_features_lr.fit(my_features_trainingData)
my_features_predictions = my_features_model.transform(my_features_testData)
my_features_predictions.select("prediction", "price", "features").show(5)
my_features_testData.select(mean('price').alias('price_value')).collect()  #544128.029162747
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(my_features_predictions) #
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse) #273356 (previous 281234)

+------------------+------+--------------------+
 prediction| price| features|
+------------------+------+--------------------+
 391979.3653415367|400000|[3.0,1.0,1460.0,4...|
 288834.1371979937|281000|[3.0,1.0,1340.0,2...|
 462290.3368849978|239000|[4.0,2.0,1980.0,1...|
204303.31686851382|150000|[2.0,1.0,840.0,12...|
476407.85367543995|617000|[3.0,2.5,1910.0,4...|
+------------------+------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 273356

In [8]:
(my_features_trainingData, my_features_testData) = home_data.randomSplit([0.8, 0.2],seed = 11L)
my_features_assembler = VectorAssembler(inputCols=my_features,outputCol="features")
my_features_lr = LinearRegression(labelCol="price", featuresCol="features")

#Pipeline

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[my_features_assembler, my_features_lr])
pipelineModel = pipeline.fit(my_features_trainingData)
predictions = pipelineModel.transform(my_features_testData)
predictions.select("price", "prediction", "features").show()
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse


+------+------------------+--------------------+
 price| prediction| features|
+------+------------------+--------------------+
400000| 391979.3653415367|[3.0,1.0,1460.0,4...|
281000| 288834.1371979937|[3.0,1.0,1340.0,2...|
239000| 462290.3368849978|[4.0,2.0,1980.0,1...|
150000|204303.31686851382|[2.0,1.0,840.0,12...|
617000|476407.85367543995|[3.0,2.5,1910.0,4...|
872750| 793821.0710093081|[3.0,2.5,2870.0,1...|
295000|431996.77676835656|[3.0,1.5,1640.0,7...|
250000| 386925.3324143812|[3.0,2.25,1640.0,...|
189000|288007.57349818945|[2.0,1.0,1200.0,9...|
550120| 613587.3266409189|[5.0,2.5,2620.0,8...|
221900|247848.30801115185|[2.0,1.0,780.0,67...|
130000| 288677.1601165086|[2.0,1.0,880.0,90...|
495000|465550.79051327705|[3.0,3.5,1650.0,1...|
311850| 435349.1141318381|[4.0,2.5,1890.0,4...|
720000| 606038.2101709768|[3.0,1.5,2180.0,5...|
324900|462571.88115768135|[4.0,2.25,2010.0,...|
830000| 989705.8757888973|[3.0,2.5,3370.0,6...|
359950| 522591.4853808433|[3.0,2.5,1980.0,7...|
750000|507496.91925381124|[4.0,1.5,2060.0,4...|
800000| 736799.4111452922|[4.0,2.5,2930.0,5...|
+------+------------------+--------------------+
only showing top 20 rows

RMSE on our test set: 273356

In [9]:
#RandomForestRegressor

from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="price", featuresCol="features")
pipeline_rf = Pipeline(stages=[my_features_assembler, rf])
pipeline_rf_Model = pipeline_rf.fit(my_features_trainingData)
predictions_rf = pipeline_rf_Model.transform(my_features_testData)
predictions_rf.select("price", "prediction", "features").show()
rmse_rf = evaluator.evaluate(predictions_rf)
print "rmse_rf on our test set: %g" % rmse_rf


+------+------------------+--------------------+
 price| prediction| features|
+------+------------------+--------------------+
400000|357579.64300964493|[3.0,1.0,1460.0,4...|
281000| 365493.0378603019|[3.0,1.0,1340.0,2...|
239000| 472862.4463503667|[4.0,2.0,1980.0,1...|
150000|328806.75709003967|[2.0,1.0,840.0,12...|
617000| 457917.1336993727|[3.0,2.5,1910.0,4...|
872750| 669133.5963161837|[3.0,2.5,2870.0,1...|
295000| 452066.9390294943|[3.0,1.5,1640.0,7...|
250000| 385433.3616160882|[3.0,2.25,1640.0,...|
189000| 261504.8603742059|[2.0,1.0,1200.0,9...|
550120| 544017.4441838029|[5.0,2.5,2620.0,8...|
221900|324557.19669065747|[2.0,1.0,780.0,67...|
130000| 321875.2760741905|[2.0,1.0,880.0,90...|
495000| 524863.5035813581|[3.0,3.5,1650.0,1...|
311850| 468448.302588876|[4.0,2.5,1890.0,4...|
720000| 564806.8485928759|[3.0,1.5,2180.0,5...|
324900| 460260.4731221936|[4.0,2.25,2010.0,...|
830000| 796112.6339838735|[3.0,2.5,3370.0,6...|
359950|414349.28208828525|[3.0,2.5,1980.0,7...|
750000| 544248.7567972663|[4.0,1.5,2060.0,4...|
800000| 756158.2074669448|[4.0,2.5,2930.0,5...|
+------+------------------+--------------------+
only showing top 20 rows

rmse_rf on our test set: 271392

In [10]:
#Gradient-boosted trees (GBTs)
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(labelCol="price", featuresCol="features")
pipeline_gbt = Pipeline(stages=[my_features_assembler, gbt])
pipeline_gbt_Model = pipeline_gbt.fit(my_features_trainingData)
predictions_gbt = pipeline_gbt_Model.transform(my_features_testData)
predictions_gbt.select("price", "prediction", "features").show()
rmse_gbt = evaluator.evaluate(predictions_gbt)
print "rmse_gbt on our test set: %g" % rmse_gbt


+------+------------------+--------------------+
 price| prediction| features|
+------+------------------+--------------------+
400000|291922.49028255447|[3.0,1.0,1460.0,4...|
281000|292285.60493684147|[3.0,1.0,1340.0,2...|
239000|369581.85305260925|[4.0,2.0,1980.0,1...|
150000|265633.75697014347|[2.0,1.0,840.0,12...|
617000| 508590.8999038055|[3.0,2.5,1910.0,4...|
872750| 722618.1397359445|[3.0,2.5,2870.0,1...|
295000| 514366.7221467832|[3.0,1.5,1640.0,7...|
250000| 244813.3320598557|[3.0,2.25,1640.0,...|
189000| 186032.8200990168|[2.0,1.0,1200.0,9...|
550120| 531065.1832313621|[5.0,2.5,2620.0,8...|
221900|241723.66842805975|[2.0,1.0,780.0,67...|
130000|256918.42888799834|[2.0,1.0,880.0,90...|
495000| 665790.2674631117|[3.0,3.5,1650.0,1...|
311850| 340805.9242203145|[4.0,2.5,1890.0,4...|
720000| 749360.2008023189|[3.0,1.5,2180.0,5...|
324900| 346615.3302100003|[4.0,2.25,2010.0,...|
830000| 955922.7045834075|[3.0,2.5,3370.0,6...|
359950|363962.35238607664|[3.0,2.5,1980.0,7...|
750000| 592359.4557916268|[4.0,1.5,2060.0,4...|
800000| 863220.3657845607|[4.0,2.5,2930.0,5...|
+------+------------------+--------------------+
only showing top 20 rows

rmse_gbt on our test set: 225391